In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
import gspread
from google.oauth2 import service_account
import json
import google.auth
from google.auth.transport.requests import AuthorizedSession
from google.oauth2.credentials import Credentials
from oauth2client.service_account import ServiceAccountCredentials



#
#
# RUN SELENIUM

In [2]:
driver = webdriver.Chrome(r"/Users/jarkrunglerdkriangkrai/Github/setscraping/chromedriver")

#
#
# GET DATA

In [3]:
driver.get('http://siamchart.com/stock/')

In [4]:
data = driver.page_source

#
#
# CONVERT TO DATAFRAME

In [5]:
pd.read_html(data)[2]

,Name,No.,Links,Sign,Last,Chg%,Volume,Value (k),MCap (M),P/E,...,ROA%,ROE%,NPM%,Yield%,FFloat%,MG%,Magic1,Magic2,PEG,CG
0,Average,NaN,NaN,NaN,19.73,1.25,5377584.93,64616.07,25666.19,36.23,...,5.48,7.09,12.06,0.09,41.39,NaN,617.88,615.33,1.96,NaN
1,24CS,1.0,i,NaN,2.88,1.41,779800.00,2259.00,0.00,NaN,...,NaN,NaN,NaN,-,30.23,NaN,NaN,NaN,NaN,NaN
2,2S,2.0,i | 1 | 2 | 3,NaN,3.24,-1.22,221300.00,718.00,1793.00,NaN,...,-2.64,-4.19,-2.68,13.19,57.50,NaN,NaN,NaN,NaN,NaN
3,3K-BAT,3.0,i | 1 | 3,NaN,62.00,0.00,0.00,0.00,4861.00,29.33,...,6.46,7.05,3.05,0.50,2.10,NaN,649.00,619.00,0.29,NaN
4,7UP,4.0,i | 1 | 2 | 3,NaN,0.78,0.00,8881400.00,6977.00,4063.00,119.96,...,0.31,1.32,-12.96,NaN,63.37,NaN,1207.00,1262.00,-4.90,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,YGG,822.0,i | 1 | 3,NaN,8.15,2.52,717700.00,5761.00,4876.00,40.08,...,23.60,24.66,36.59,0.86,45.15,NaN,567.00,517.00,0.49,NaN
823,YONG,823.0,i,NaN,2.20,1.85,3278900.00,7218.00,1523.00,17.20,...,9.49,11.42,9.25,NaN,27.24,NaN,538.00,488.00,NaN,NaN
824,YUASA,824.0,i | 1 | 3,NaN,15.20,0.00,445600.00,6854.00,1636.00,14.95,...,8.95,11.03,4.15,2.28,15.27,NaN,518.00,476.00,-24.83,NaN
825,ZEN,825.0,i | 1 | 3,NaN,17.30,1.17,379700.00,6474.00,5190.00,47.83,...,7.31,9.29,4.02,NaN,27.34,NaN,856.00,808.00,-2.77,NaN


In [6]:
df = pd.read_html(data)[2].drop(['No.','Links','CG','Sign','MG%'],axis=1).tail(-1)
df

,Name,Last,Chg%,Volume,Value (k),MCap (M),P/E,P/BV,D/E,DPS,EPS,ROA%,ROE%,NPM%,Yield%,FFloat%,Magic1,Magic2,PEG
1,24CS,2.88,1.41,779800.0,2259.0,0.0,NaN,-,NaN,-,0.00,NaN,NaN,NaN,-,30.23,NaN,NaN,NaN
2,2S,3.24,-1.22,221300.0,718.0,1793.0,NaN,0.90,0.26,0.27,0.00,-2.64,-4.19,-2.68,13.19,57.50,NaN,NaN,NaN
3,3K-BAT,62.00,0.00,0.0,0.0,4861.0,29.33,2.04,0.61,0.06,0.54,6.46,7.05,3.05,0.50,2.10,649.0,619.0,0.29
4,7UP,0.78,0.00,8881400.0,6977.0,4063.0,119.96,1.61,0.38,NaN,0.01,0.31,1.32,-12.96,NaN,63.37,1207.0,1262.0,-4.90
5,A,5.00,-0.99,1900.0,10.0,4880.0,NaN,1.54,3.42,NaN,0.00,2.11,-2.60,2.87,NaN,24.92,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,YGG,8.15,2.52,717700.0,5761.0,4876.0,40.08,8.02,0.05,0.08,0.20,23.60,24.66,36.59,0.86,45.15,567.0,517.0,0.49
823,YONG,2.20,1.85,3278900.0,7218.0,1523.0,17.20,1.90,0.63,NaN,0.13,9.49,11.42,9.25,NaN,27.24,538.0,488.0,NaN
824,YUASA,15.20,0.00,445600.0,6854.0,1636.0,14.95,1.60,0.59,0.35,1.02,8.95,11.03,4.15,2.28,15.27,518.0,476.0,-24.83
825,ZEN,17.30,1.17,379700.0,6474.0,5190.0,47.83,4.24,1.10,NaN,0.36,7.31,9.29,4.02,NaN,27.34,856.0,808.0,-2.77


In [7]:
import pickle
file_name='my_file.pkl'
f = open(file_name,'wb')
f.close()

In [8]:
df.to_pickle('/Users/jarkrunglerdkriangkrai/Github/setscraping/my_file.pkl')


#
#
# UPDATE TO GOOGLE SHEETS

In [9]:
import gspread
gc = gspread.service_account(filename='/Users/jarkrunglerdkriangkrai/Github/setscraping/service_account.json')

sh = gc.open("set scraping")


In [10]:
wks = sh.worksheet("Sheet1")

In [11]:
print('Rows: ', wks.row_count)
print('Columns: ', wks.col_count)

Rows:  1000
Columns:  26


In [12]:
#creds, project = google.auth.default(scopes=['https://www.googleapis.com/auth/spreadsheets'])
#service = build('sheets', 'v4', credentials=creds)

In [13]:
wks.clear()

{'spreadsheetId': '1sDLJJubzeMhc_U4xUe-P3rvSF7t7pV8tRKeabtux6WQ',
 'clearedRange': 'Sheet1!A1:Z1000'}

In [14]:
#wks.update('A1',df)

TypeError: Object of type DataFrame is not JSON serializable

In [ ]:
set_with_dataframe(worksheet='Sheet1', dataframe=df)

In [ ]:
#finaldata = data.values.tolist()


In [ ]:
wks.update('A1',df)

{'spreadsheetId': '1sDLJJubzeMhc_U4xUe-P3rvSF7t7pV8tRKeabtux6WQ',
 'updatedRange': 'Sheet1!A1'}